In [1]:
from sentence_transformers import SentenceTransformer
import torch
torch._dynamo.config.suppress_errors = True
# model = SentenceTransformer('answerdotai/ModernBERT-large')
model = SentenceTransformer("all-mpnet-base-v2")
MODEL_NAME = "all-mpnet-base-v2"

In [2]:
import sqlite3
from copy import deepcopy
import json

DATABASE_NAME = "suika_commands.db"
TABLE = "suika_commands"

def fetch_all_documents():
    try:
        conn = sqlite3.connect(DATABASE_NAME)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {TABLE}")
        rows = cursor.fetchall()
        documents = [dict(row) for row in rows]
        for doc in documents:
            doc["keywords"] = json.loads(doc["keywords"])
            doc["examples"] = json.loads(doc["examples"])
    except sqlite3.OperationalError as e:
        print(f"OperationalError: {e}")
        documents = []
    finally:
        conn.close()
    return documents

In [3]:
all_docs = fetch_all_documents()
print('total', len(all_docs))
print(all_docs[0])

total 1195
{'linux_cmd_id': 'linux_pw-link', 'name': 'pw-link', 'description': 'Pw-link is a command-line tool for Linux that manages symbolic links specifically for password files encrypted with the FileVault system.', 'syntax': 'pw-link [-d] [options] <source> <destination>', 'keywords': ['link', 'symbolic', 'password', 'FileVault', 'source', 'destination', 'encryption', 'remote'], 'examples': ['pw-link /var/lib/vault.root /Users/username/Documents', 'pw-link -d /tmp/vault.pw /Users/username/Vault']}


In [4]:
import numpy as np
import faiss
from tqdm import tqdm

embed_docs = []
id_to_vec_pos = {}
for i,doc in tqdm(enumerate(all_docs)):
    emb = model.encode(doc['description'])
    embed_docs.append(emb)
    id_to_vec_pos[doc['linux_cmd_id']] = i
embed_docs = np.vstack(embed_docs)

embed_docs.shape

1195it [00:21, 54.87it/s]


(1195, 768)

In [5]:
d = embed_docs.shape[1]
# index = faiss.IndexFlatL2(d)
index = faiss.IndexFlatIP(d)
index.add(embed_docs)
index.ntotal

1195

In [19]:
k = 1
q = "how to copy the file in linux?"
qemb = model.encode(q)
qemb = qemb[None]
qemb.shape

(1, 768)

In [20]:
%%time
D, I = index.search(qemb, k)
print(all_docs[I[0].item()])
print(D)

{'linux_cmd_id': 'linux_cp', 'name': 'cp', 'description': 'Copy files and directories from one location to another. Uses syntax similar to mv, often but not always more efficient than mv when copying large files.', 'syntax': 'cp [options] source destination', 'keywords': ['copy', 'duplicate', 'files', 'directories', 'source', 'destination'], 'examples': ['cp myfile.txt backup.txt', 'cp -r Documents/ new_location/']}
[[0.40737653]]
CPU times: user 1.41 ms, sys: 157 μs, total: 1.57 ms
Wall time: 970 μs


In [8]:
index_file = "suika_commands_vector.index"
faiss.write_index(index, index_file)

In [9]:
index = faiss.read_index(index_file)

In [10]:
%%time
D, I = index.search(qemb, k)
print(all_docs[I[0].item()])
print(D)

{'linux_cmd_id': 'linux_cp', 'name': 'cp', 'description': 'Copy files and directories from one location to another. Uses syntax similar to mv, often but not always more efficient than mv when copying large files.', 'syntax': 'cp [options] source destination', 'keywords': ['copy', 'duplicate', 'files', 'directories', 'source', 'destination'], 'examples': ['cp myfile.txt backup.txt', 'cp -r Documents/ new_location/']}
[[0.40737653]]
CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 1.54 ms


In [11]:
import sqlite3
conn = sqlite3.connect(DATABASE_NAME)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS id_to_vec_pos (
        faiss_index_id INTEGER PRIMARY KEY,
        linux_cmd_id TEXT NOT NULL
    )
''')
conn.commit()
conn.close()

In [14]:
def save_mapping_to_db(database_name, mapping):
    # use context manager to ensure connection and cursor are properly closed
    with sqlite3.connect(database_name) as conn:
        cursor = conn.cursor()
        for linux_cmd_id, faiss_index_id in mapping.items():
            cursor.execute('''
                INSERT OR REPLACE INTO id_to_vec_pos (faiss_index_id, linux_cmd_id)
                VALUES (?, ?)
            ''', (faiss_index_id, linux_cmd_id))  # replaces if faiss_index_id already exists
        conn.commit()

save_mapping_to_db(DATABASE_NAME, id_to_vec_pos)

In [1]:
from sentence_transformers import SentenceTransformer
import torch
torch._dynamo.config.suppress_errors = True

model = SentenceTransformer("all-mpnet-base-v2")

In [2]:
import vec_db as _vec_db

q_emb = model.encode('how to query db')
_vec_db.search(q_emb)

doc_id linux_mycli


({'linux_cmd_id': 'linux_mycli',
  'name': 'mycli',
  'description': 'mycli is a modern, user-friendly command-line interface for interacting with MySQL databases. It provides an intuitive way to execute SQL queries, manage schema objects, and analyze data.',
  'syntax': 'mycli [options] [database]',
  'keywords': '["mysql", "database", "sql", "query", "management", "command-line"]',
  'examples': '["mycli", "mycli --database testdb", "mycli -h localhost -u user -p mydb"]'},
 0.38226398825645447)